In [25]:
# Import libraries
import pandas as pd

# Load hourly data
hourly_data = pd.read_csv('Calgary_merged_data.csv', parse_dates=['DateTime'])

# Load 2A data
daily_data = pd.read_csv('AESO/Aeco 2A.csv', parse_dates=['Trade Date'])

In [26]:
daily_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3354 entries, 0 to 3353
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Trade Date              3354 non-null   datetime64[ns]
 1   Index                   3345 non-null   object        
 2   Trade Start             3345 non-null   object        
 3   Trade End               3345 non-null   object        
 4   Delivery Start          3345 non-null   object        
 5   Delivery End            3345 non-null   object        
 6   Index Price             3345 non-null   float64       
 7   Price Currency          3345 non-null   object        
 8   Duration                3345 non-null   float64       
 9   Traded Volume           3345 non-null   float64       
 10  Total Traded Volume     3345 non-null   float64       
 11  Traded Volume Unit      3345 non-null   object        
 12  Alternate Volume        0 non-null      float64 

In [27]:
# Drop not needed columns from 2A data
daily_data.drop(columns=['Index','Trade Start','Trade End','Delivery Start','Delivery End','Price Currency', 'Traded Volume', 'Total Traded Volume','Traded Volume Unit','Alternate Volume','Total Alternate Volume','Duration','Trades In Index','Last Update Time','Component Type','Settle State'], inplace=True)


In [28]:
daily_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3354 entries, 0 to 3353
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Trade Date   3354 non-null   datetime64[ns]
 1   Index Price  3345 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 52.5 KB


In [29]:
# Rename some columns to clean the data
daily_data.rename(columns={'Trade Date': 'DateTime'}, inplace=True)

# Expand the daily data into 24 hours
daily_expanded = daily_data.loc[:, ['DateTime', 'Index Price']].copy()
daily_expanded['DateTime'] = daily_expanded['DateTime'].dt.floor('D')  # Truncate to date
daily_expanded = daily_expanded.set_index('DateTime').resample('H').ffill().reset_index()

# Merge the DataFrames based on 'DateTime'
merged_df = pd.merge(hourly_data, daily_expanded, on='DateTime', how='inner')

# Drop the extra 'Date' column created by the merge
merged_df.drop(columns=['Date'], inplace=True)

# Rename column
merged_df.rename(columns={'Index Price': 'NG_price'}, inplace=True)

# Save the merged DataFrame to a new file
merged_df.to_csv('merged_expanded_data.csv', index=False)
print("Daily data has been expanded into 24 hours and merged with hourly data.")

C:\Users\gaoqian\AppData\Local\Temp\ipykernel_47156\1095266215.py:7: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  daily_expanded = daily_expanded.set_index('DateTime').resample('H').ffill().reset_index()


Daily data has been expanded into 24 hours and merged with hourly data.
